# Predicting requests

## Loading data

The data shows hourly requests cumulatively over five weeks, starting on Monday.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(1)

#Load
requests = pd.read_csv('requests_every_hour.csv',header=0)

#Overview
print(requests.dtypes)
requests.head()


## Analysing data

The first step is to explore what the data looks like. Are there cyclical patterns, seasonal patterns, trends?

In [ ]:
#one day
plt.figure(figsize=(20,5)).suptitle("Day", fontsize=20)
plt.plot(requests.head(24))
plt.show()

#one week
plt.figure(figsize=(20,5)).suptitle("Week", fontsize=20)
plt.plot(requests.head(168))
plt.show()

#overall
plt.figure(figsize=(20,10)).suptitle("Overall", fontsize=20)
plt.plot(requests)
plt.show()

### Excursus: autocorrelation

In time series analysis, autocorrelation can provide information about the extent to which previous values influence the next x values (a = number of lags). Here, we can also see the periodic/seasonal dependency of the values in the correlation.</br>
https://machinelearningmastery.com/gentle-introduction-autocorrelation-partial-autocorrelation/

In [ ]:
# Import plot function
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Autocorrelation plot
plot_acf(requests['Requests'], lags = 48)
plt.show()

In [ ]:
# Partial autocorrelation plot
plot_pacf(requests['Requests'], lags = 48, method='ols')
plt.show()

#### Example: Periodic data with and without noise

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Generate a sinusoidal time series without noise
np.random.seed(42)
date_rng = pd.date_range(start='2023-01-01', end='2023-01-31', freq='H')  # Hourly data for a month
sinusoidal_pattern = np.sin(2 * np.pi * date_rng.hour / 24) * 20
df = pd.DataFrame(data={'datetime': date_rng, 'sinusoidal': sinusoidal_pattern})

# Set the 'datetime' column as the index
df.set_index('datetime', inplace=True)

fig, ax = plt.subplots(figsize=(10, 4))
plt.title('Raw data')
plt.plot(df)
plt.show()

# Autocorrelation plot
fig, ax = plt.subplots(figsize=(10, 4))
sm.graphics.tsa.plot_acf(df['sinusoidal'], lags=48, ax=ax)  # Assuming hourly data, so lags=24 for 1-day period
plt.title('Autocorrelation Plot')
plt.show()

# Partial autocorrelation plot
fig, ax = plt.subplots(figsize=(10, 4))
sm.graphics.tsa.plot_pacf(df['sinusoidal'], lags=48, method='ols', ax=ax)
plt.title('Partial Autocorrelation Plot')
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Generate a strongly seasonal time series dataset
np.random.seed(42)
date_rng = pd.date_range(start='2023-01-01', end='2023-01-31', freq='H')  # Hourly data for a month
daily_seasonality = np.sin(2 * np.pi * date_rng.hour / 24) * 20
sales = 100 + daily_seasonality + np.random.normal(0, 5, size=len(date_rng))

df = pd.DataFrame(data={'date': date_rng, 'sales': sales})

# Set the 'date' column as the index
df.set_index('date', inplace=True)

fig, ax = plt.subplots(figsize=(10, 4))
plt.title('Raw data')
plt.plot(df)
plt.show()


# Autocorrelation plot
fig, ax = plt.subplots(figsize=(10, 4))
sm.graphics.tsa.plot_acf(df['sales'], lags=40, ax=ax)
plt.title('Autocorrelation Plot')
plt.show()

# Partial autocorrelation plot
fig, ax = plt.subplots(figsize=(10, 4))
sm.graphics.tsa.plot_pacf(df['sales'], lags=40, ax=ax, method='ols')
plt.title('Partial Autocorrelation Plot')
plt.show()


## Prepare data for LSTM

In [ ]:
from sklearn.preprocessing import StandardScaler

#Scale data
print("Enquiries Range before scaling:" , 
          min(requests.Requests),
          max(requests.Requests))

scaler = StandardScaler()
scaled_requests=scaler.fit_transform(requests)
print("Enquiries Area after scaling:" , 
          min(scaled_requests),
          max(scaled_requests))

#It is important to map periodic signals with full periods.
train_size = 24 * 7 * 4 #672

#How far back do we want to look? One week here
lookback = 24 * 7  #168

#We can't split this up at random, otherwise the temporal context will be lost.
#So we take the first 4 weeks as training data --> 672 data points.
train_requests = scaled_requests[0:train_size,:]

#Test data is the rest of the data plus the last week of training data to make predictions
test_requests = scaled_requests[train_size-lookback:,:]

print("\nForm von scaled_request : ", scaled_requests.shape)
print("\nForm von train_requests, test_requests : ",
      train_requests.shape, test_requests.shape)

In [ ]:
#Helper function to prepare the data set for the LSTM
#Each data point (X) is linked to the previous data points of size=lookback.
#The predicted value (Y) is the next point.
def create_lstm_dataset(data, lookback = 1):
    #create two empty lists
    data_x, data_y = [], []
    for i in range(len(data) - lookback - 1):
            #All points from this point backwards for the lookback period
            a = data[i:(i + lookback), 0]
            data_x.append(a)
            #next datapoint
            data_y.append(data[i + lookback, 0])
    return np.array(data_x), np.array(data_y)

#Create X and Y for training
train_req_x, train_req_y = create_lstm_dataset(train_requests, lookback)

print("Form von train_req_x, train_req_y: ",train_req_x.shape, train_req_y.shape)

#Transform training data for LSTM
train_req_x = np.reshape(train_req_x, (train_req_x.shape[0],1, train_req_x.shape[1]))

print("Form von train_req_x, train_req_y: ",train_req_x.shape, train_req_y.shape)

## Model the actual LSTM with Keras

In [ ]:
#maybe you need to install tensorflow
#!pip install tensorflow --user

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM,Dense
import tensorflow as tf

tf.random.set_seed(3)

#Create model
ts_model=Sequential()

#Add LSTM layer
ts_model.add(LSTM(256, input_shape=(1,lookback)))
#Output layer --> condense to 1 output
ts_model.add(Dense(1))

#Compiling with Adam Optimizer. Optimising for minimum mean square error
ts_model.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

#Output model
ts_model.summary()

#Train model
ts_model.fit(train_req_x, train_req_y, epochs=10, batch_size=1, verbose=1)

## Background info:

### Batch size: </br>
The batch size is a hyperparameter that determines the number of samples that are run before the internal model parameters are updated.

Think of a batch as a for loop that iterates over one or more samples and makes predictions. At the end of the batch, the predictions are compared to the expected output variables and an error is calculated. This error is used by the update algorithm to improve the model.

A training dataset can be divided into one or more batches.

### Epoch: </br>
The number of epochs is a hyperparameter that determines how many times the learning algorithm will go through the entire training data set.

One epoch means that each sample in the training data set has had a chance to update the internal model parameters. An epoch consists of one or more batches. For example, one epoch with one batch is called batch gradient descent learning algorithm.

### What is the difference between batch and epoch?</br>
The batch size is the number of samples processed before the model is updated.

The number of epochs is the number of complete passes through the training data set.

The batch size must be greater than or equal to one and less than or equal to the number of samples in the training data set.

The number of epochs can be set to any integer value between one and infinity. You can let the algorithm run for as long as you like and even stop it based on criteria other than a fixed number of epoch, such as a change (or lack thereof) in the model error over time.</br></br>
<a href=https://machinelearningmastery.com/difference-between-a-batch-and-an-epoch>https://machinelearningmastery.com/difference-between-a-batch-and-an-epoch</a>


## Test model

In [ ]:
#Prepare the test data set in the same way as the training data set.
test_req_x, test_req_y = create_lstm_dataset(test_requests,lookback)
test_req_x = np.reshape(test_req_x, 
                         (test_req_x.shape[0],1, test_req_x.shape[1]))

#Evaluate model
ts_model.evaluate(test_req_x, test_req_y, verbose=1)

#Calculate prediction for training data set
predict_on_train= ts_model.predict(train_req_x)
#Calculate prediction for test data set
predict_on_test = ts_model.predict(test_req_x)

#Rescale data to compare with original
predict_on_train = scaler.inverse_transform(predict_on_train)
predict_on_test = scaler.inverse_transform(predict_on_test)


## Plot output

In [ ]:
#Total size x-axis
total_size = len(predict_on_train) + len(predict_on_test)

#Original data 
orig_data=requests.Requests.to_numpy()
#Reformatting in (number of points, 1) for the plot
orig_data=orig_data.reshape(len(orig_data),1)
#Create an ‘empty’ array for plotting
orig_plot = np.empty((total_size,1))
#Init array
orig_plot[:, :] = np.nan
#Transfer data
orig_plot[0:total_size, :] = orig_data[lookback:-2,]

#Data for training forecasts.
predict_train_plot = np.empty((total_size,1))
predict_train_plot[:, :] = np.nan
predict_train_plot[0:len(predict_on_train), :] = predict_on_train

#Data for test forecasts.
predict_test_plot = np.empty((total_size,1))
predict_test_plot[:, :] = np.nan
predict_test_plot[len(predict_on_train):total_size, :] = predict_on_test

#Output all data
plt.figure(figsize=(20,10)).suptitle("Predictions for original, training and test data", fontsize=20)
plt.plot(orig_plot, label="Original")
plt.plot(predict_train_plot, label="Training")
plt.plot(predict_test_plot,label="Test")
plt.legend()
plt.show()

## Predicting the future

The model created can predict the next value in the time series if we provide the previous 168 actual values. This means that it can only predict the first hour of next week. To predict the second hour, we need the actual value for the first hour. So how can we predict a whole week? 
<p>
To account for the missing value in the look-back sequence, we can use the predicted value in the look-back. So for the first hour's prediction, P1, we use the last 168 values of the actual data. For the next hour, P2, we then use our first prediction P1 together with the last 167 values of the actual data. This way, we get contiguous values for the previous sequence, even when one of them is predicted. We can continue along the same lines, adding the current prediction to the look-back for the next prediction.</p><p>
Note that as you use more and more prediction instead of actual values in the input, the output will become less and less accurate and after a certain time the patterns can no longer be maintained. It is therefore recommended not to make predictions for a future sequence whose size is greater than the size of the lookback. In this case, both are one week. </p>

| Prediction | Lookback |
|:--------------|:-------------------------|
| P1 | A168 - A1 |
| P2 | P1, A168 - A2 |
| P3 | P2 - P1, A168 - A3 |
| P4 | P3 -P1, A168 - A4 |
| P5 | P4 - P1, A168 - A5 |

<p>
The following code block implements the logic for creating lookbacks with predictions. 
Then, we perform an inverse transform to scale the values back. Finally, we plot training, test and future prediction in a single diagram. The orange line shows the predictions for four weeks of training data. The green line shows the predictions for one week of test data. The red line refers to one week in the future, using the predicted values and lookbacks. </p>

In [ ]:
#Use the last part of the training data as the first review
curr_input= test_req_x[-1,:].flatten()

#Forecast for next week
predict_for = 24 * 7

for i in range(predict_for):
    
    #Let X be the number of the last lookbacks.
    this_input=curr_input[-lookback:]
    #Generate input
    this_input=this_input.reshape((1,1,lookback))
    #Predict next datapoint
    this_prediction=ts_model.predict(this_input)

    #Add the current prediction to the input
    curr_input = np.append(curr_input,this_prediction.flatten())
    
#Extract the last part of predict_for from curr_input that contains all new predictions.
predict_on_future=np.reshape(np.array(curr_input[-predict_for:]),(predict_for,1))

#Scale back
predict_on_future=scaler.inverse_transform(predict_on_future)

#show the next ten datapoints
print(predict_on_future[:10])

In [ ]:
#Plot the training data with the forecast data
total_size = len(predict_on_train) + len(predict_on_test) + len(predict_on_future)

#Setup training chart
predict_train_plot = np.empty((total_size,1))
predict_train_plot[:, :] = np.nan
predict_train_plot[0:len(predict_on_train), :] = predict_on_train

#Setup test chart
predict_test_plot = np.empty((total_size,1))
predict_test_plot[:, :] = np.nan
predict_test_plot[len(predict_on_train):len(predict_on_train)+len(predict_on_test), :] = predict_on_test

#Setup future forecast chart
predict_future_plot = np.empty((total_size,1))
predict_future_plot[:, :] = np.nan
predict_future_plot[len(predict_on_train)+len(predict_on_test):total_size, :] = predict_on_future

plt.figure(figsize=(20,10)).suptitle("Predictions for original, training and test data", fontsize=20)
plt.plot(orig_plot, label="Original")
plt.plot(predict_train_plot, label="Training")
plt.plot(predict_test_plot,label="Test")
plt.plot(predict_future_plot,label="Future")
plt.legend()
plt.show()